<img src="docs/ajccbc.png">

<p></p>
<h1>Follow the money!</h1>

*by Eireann Leverett and Serge Droz*

Version: Oct 20, 2023

This lesson will focus on ransomware and cryptocurrencies to introduce a some basic data science and statistical concepts. We'll look at a campaign called *Deadbolt*, and do some **survival analysis**. This term describes statistical methods that try to determine the duration of certain events. The name derives from biology where a common questions is how long does a population survive after a given event. The obvious question in incident response is how much time does it take from an initial compromise to full encryption. 


We then learn a surprising decryptor key mechanism. 

This tutorial is inspired by the good work we did in collaboration with Mark Ellzey @ censys.io. When you've finished the tutorial, go read some of the blogs listed at the bottom of this Lesson. This is possible, because there is a lot of information available about the ransomware. 

#### License

This material is available under the [Creative Commons BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

In [ ]:
import pandas as pd
import json
from scipy.stats import pearsonr
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
import numpy as np
from lifelines import KaplanMeierFitter, NelsonAalenFitter, WeibullFitter
from lifelines.utils import median_survival_times, datetimes_to_durations

We are going to analyse data about crypto payments related to the deadbold ransomware. The csv file contains transactions, i.e. the date, amount and wallets.

In [ ]:
#Get the data on all ransoms paid
family_ransoms = pd.read_csv('data/DeadBolt_Accounts_RANSOMS.csv',index_col='confirmed_datetime',
                              parse_dates=['confirmed_datetime','received_datetime'],
                              date_format='ISO8601', low_memory=False)
family_ransoms.info()


Since we're interested in analyzing temporal behavior we sort the data frame after the received time, which is closer to the infection date that the confirmed time, which indicates when the transaction completed. So here we make an assumption, namely that the received time somehow correlates with the infection time. That's the best we can do without having to analyze each of the incidents in detail. This is a plausible assumption, but bey no means solid. 

In [ ]:
family_ransoms = family_ransoms.sort_values('received_datetime')

Mark provided a json file containing the number of infections at a given date.

In [ ]:
#Infection data from Censys.io
deadbolt_infections = [
  { "d": "2022-01-24", "count":    "1" },
  { "d": "2022-01-25", "count": "4142" },
  { "d": "2022-01-26", "count": "4988" },
  { "d": "2022-01-27", "count": "3927" },
  { "d": "2022-01-28", "count": "2002" },
  { "d": "2022-01-29", "count": "1128" },
  { "d": "2022-01-30", "count":  "920" },
  { "d": "2022-01-31", "count":  "853" },
  { "d": "2022-02-01", "count":  "783" },
  { "d": "2022-02-02", "count":  "735" },
  { "d": "2022-02-03", "count":  "712" },
  { "d": "2022-02-04", "count":  "656" },
  { "d": "2022-02-05", "count":  "627" }
]

Convert this to a dataframe (what else?):

In [ ]:
infection_timeseries = pd.DataFrame( deadbolt_infections,  )
infection_timeseries.rename( { 'd' : 'Date' , 'count' : 'Count'} , inplace=True,axis=1)
infection_timeseries['Date'] = pd.to_datetime( infection_timeseries['Date'] )
infection_timeseries = infection_timeseries.astype( {'Count': 'int64'})

infection_timeseries.set_index('Date', inplace=True)
infection_timeseries.head()

## A crash course in statistics

Now let's do a little science and test something that should be intuitive. Does the number of infections correlate to the number of ransoms paid (numerically speaking). We'll need to talk a little about **the null hypothesis** is that they do not.

In statistics the **the null hypothesis $H_0$** is the claim that two variables are not correlated, i.e. their relationship is truly random. For example a given drug has zero effect on some medical condition. Thus out job is to show that the null hypothesis is wrong.

### Correlation

In statistics **correlation** is a number indicating how strong the relation between two data sets is. There are different definitions but the most common one is **Pearson coefficient**, a number between -1 and 1. ! means positive correlation, i.e. if one number grows so does the other, and -1 is negative correlation indication that as one number grows the other decreases. For example height correlates with weight. A correlation of 0 says the two number do not correlate.

 > Correlation does not imply causation! For example the number of storks and the number of births are correlated: [Do Storks Deliver Babies?](https://priceonomics.com/do-storks-deliver-babies/)

For completeness sake the formula for the Pearson correlation of a dataset $[ (x_1, y_1), ... (x_n, y_n)]$ is
$$
  r = \frac{\Sigma ( x_i - \bar{x})(y_i - \bar{y} )}{\sqrt{ \Sigma ( x_i - \bar{x})^2 \Sigma ( y_i - \bar{y} )^2 } }  = \frac{n ( \Sigma x_i y_i ) - (\Sigma x_i)(\Sigma y_i)}{ \sqrt{ (  n \Sigma x_i^2 - (\Sigma x_i)^2 ) (  n \Sigma y_i^2 - (\Sigma y_i)^2 ) }}
$$

where the sum runs from 1 to n and $\bar{x}$ denotes the mean. Don't worry too much about the actual formula though. 

### p-value

But how much does the r value mean? If you only have only few data points anything is possible. This is where the **p-value** enters the game, it tells you how likely the given data set rejects the null hypothesis, i.e. how what the likely hood is that the given correlation is not just a coincidence. The definition of the p-value is 

> *the probability that the data set has the given correlation assuming the $H-0$ is true.  

Thus the lower the p-value the more likely the correlation is true. Typically people are happy with a p-value of less than 0.05 or 0.01 (5% or 1%).

The article [Correlation and P value](https://dataschool.com/fundamentals-of-analysis/correlation-and-p-value/) gives a nice introductions. 

Luckily the `scipy` package does all the work for us. 

To summarize:

- A low p-value **rejects the null hypothesis**
- The correlation $r$ tells us **how strongly** the data is correlated.

Let us now apply this to our data.  `family_ransoms` contains a list of ransowmare events, so we have to bin then. the Pandas function `resample` dies this for us, with the parameter `D` indicating to create bins of one day. The infection_time series only has the first 13 days, so we just cut the data of there. 

Then we feed the whole thing to `pysci.pearsonr` et voila:


In [ ]:
#Note this is a one day lag correlation
corr, p_value = pearsonr(infection_timeseries['Count'], family_ransoms['count'].resample('D').sum().head(13))
print('Pearsons correlation: %.3f' % corr)
print('P-value is: %.10F' % p_value)

Trying to sum up our result into a single scientific sentence:
We can reject the null hypothesis that ransoms don't correlate with infections.
Or in more plain English:
There's less than a 1% chance of our scientific result occuring by accident.
So, More infections = more ransoms paid.

In [ ]:
#Let's look at how the infections change over time
fig, ax = plt.subplots() 
ax.plot(infection_timeseries.index,infection_timeseries['Count'], label = "Infections")
ax.tick_params(axis='x', labelrotation = 45)
ax.set_title("Number of infectiosn over time")
ax.set_xlabel( "Date" )
ax.set_ylabel("Number")
ax.grid()


Sometimes you need a certain number over and over again. So just define it at one place, but don't forget to change it if you get new data, i.e. you learn about new infections. In our example the number of days for which we have data is such a number. It has no statistical meaning, it's just convenience here. 

In [ ]:
num_days = len(family_ransoms['count'].resample('D').sum())
num_days

Let's have a look at the ransoms payed as time goes on

In [ ]:
#Let's look at ransoms paid over time
fig, ax = plt.subplots() 
ax.plot(np.arange(1,num_days,1),family_ransoms.resample('D').count()['count'][1:], label = "Ransoms")
ax.set_xlabel('Days since the first infection')
ax.set_ylabel('Ransoms paid per day')
ax.grid()
_ = ax.set_title("Ransoms paid over time")

## Payments over time: Survival analysis

We now want to get an idea how long it takes people to pay the ransom. We don't really know when exactly people got infected. But saw above that most infections occurred with two weeks of the initial one. However ransoms were payed for nearly 500 days since that initial date. So it's probably an ok to just assume they all got infected on the first day.

We can then calculate the time a victim took if it paid. This is called a survival analysis, a concept coming from biology or medicine. Think of patients dying after a certain amount of time. The **[Kaplan–Meier estimator](https://en.wikipedia.org/wiki/Kaplan%E2%80%93Meier_estimator)** is a statistical tool to calculate the **survival function**. It gives the the percent interval of patents dying after a given amount of time, resp. the number of victims paying the ransom after a given time having been encrypted. 

The python packet [lifelines](https://lifelines.readthedocs.io) provides all the tools needed as well as further information. 

So let's calculate this. FIRST we construct a new dataframe containing two columns: `infection`, which is just the first day infections were observed and `payments` which contains the payment date or non, if no ransom was paid.

In [ ]:
first_infection = infection_timeseries.iloc[0].name.date()
max_daily_infections = max(infection_timeseries['Count'])

print("Maximum daily infections", max_daily_infections )
payment_list = family_ransoms['received_datetime'].dt.date.to_list()
non_payment_list = [None]*(max_daily_infections-len(payment_list))
full_list = payment_list + non_payment_list
#d = {'infection' : ['2022-01-24']*max_daily_infections,
d = {'infection' : [first_infection]*max_daily_infections,
     'payment'   : full_list}
survival_analysis = pd.DataFrame(d)
survival_analysis

We no convert this to durations (i.e. how many days until a ransom was paid) and a list of bools indication if a ransom was paid or not. This can then be feed to the Kaplan-Meier Fitter.  

In [ ]:
kmf = KaplanMeierFitter()
#survival_analysis = survival_analysis.reset_index()
#survival_analysis = survival_analysis[['infection','payment']]
T, E = datetimes_to_durations(survival_analysis['infection'], survival_analysis['payment'], freq='D')
kmf.fit(T, E, label='Deadbolt Ransom Payment Proportion Over Time')
ax = kmf.plot()
ax.set_xlabel('Duration from infection to ransom payment (days)')
ax.set_ylabel('Proportion of infection who did not pay ransom')
ax.grid()

The Kaplan-Meier Filter gives a confidence interval. 

In [ ]:
kmf.confidence_interval_

## Interesting Bitcoin addresses

There's a very interesting BTC address in here, that both sends and recieves. Can you figure out which one?

In [ ]:
!tail -n 2 'data/DeadBolt_Accounts_TRXNS.csv'

In [ ]:
# Get the data on all transactions
# There's a very interesting BTC address in here, that both sends and recieves. Can you figure out which one?
find_the_interesting_address= pd.read_csv('data/DeadBolt_Accounts_TRXNS.csv',index_col='confirmed_datetime',parse_dates=['confirmed_datetime','received_datetime'],low_memory=False)
find_the_interesting_address.head()


Now you may need to look at other sources than just this notebook to figure out what is going on.  Use the address in a blockchain explorer, and see if you can understand what is in the the OP_RETURN fields of  transactions to that address.

In [ ]:
# Your solution here

## More literature about Deadbolt
Mark's [initial post](https://censys.com/tracking-deadbolt-ransomware-across-the-globe/), where he contacted me to follow the money.
Mark's [follow up](https://censys.com/the-neverending-story-of-deadbolt/) work with his own data science team.
The data science team at Censys made it easy to [track infections](https://lookerstudio.google.com/reporting/f8d38b6c-9997-4bba-be93-19cf57d7371a/page/p_dxblu6m2uc). Which is a practical and real world example of how jupyter notebooks and collaboration helps incident response.